In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

import os
from PIL import Image

In [ ]:
#downloading data
!pip install -q gdown
!gdown --id 19Iv_ln4_JRdf_A_IcKQdDmN6Sp2HH3hu
!unzip dog_classifier.zip

In [ ]:
#data cleaning
path = '/content/train'

for folder in os.listdir(path):
    for img_file in os.listdir(os.path.join(path,folder)):
        img_file = os.path.join(path, folder, img_file)

        try:
            img = Image.open(img_file)
            if img.mode != 'RGB':
                os.remove(img_file)
        except:
            os.remove(img_file)


In [ ]:
path = '/content/test'
for folder in os.listdir(path):
    for img_file in os.listdir(os.path.join(path,folder)):
        img_file = os.path.join(path, folder, img_file)

        try:
            img = Image.open(img_file)
            if img.mode != 'RGB':
                os.remove(img_file)
        except:
            os.remove(img_file)

In [ ]:
#data pre processing
transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(244),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
    ])

train_set = datasets.ImageFolder('/content/train' ,  transform = transform)
test_set = datasets.ImageFolder('/content/test' ,  transform = transform)

train_set = DataLoader(dataset = train_set , batch_size=6000, shuffle=True)
test_set = DataLoader(dataset = test_set , batch_size=6000, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('using device:' , device)

using device: cuda


In [ ]:
#building model

#CNN model

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)

        self.conv1 = nn.Conv2d(in_channels = 3,out_channels =6, kernel_size =4)
        self.conv2 = nn.Conv2d(in_channels= 6,out_channels =12, kernel_size =4)
        self.conv3 = nn.Conv2d(in_channels = 12,out_channels =14, kernel_size =4)
        self.conv4 = nn.Conv2d(in_channels = 14,out_channels =16, kernel_size =4)
        self.conv5 = nn.Conv2d(in_channels = 16,out_channels =20, kernel_size =4)

        self.fc1 = nn.Linear(in_features = 20*4*4 , out_features = 250)
        self.fc2 = nn.Linear(in_features = 250 , out_features = 200)
        self.fc3 = nn.Linear(in_features = 200 , out_features = 50)
        self.fc4 = nn.Linear(in_features = 50 , out_features = 10)
        self.fc5 = nn.Linear(in_features = 10 , out_features = 2)

    def forward(self,x):
        x= self.pool(f.relu(self.conv1(x)))
        x= self.pool(f.relu(self.conv2(x)))
        x= self.pool(f.relu(self.conv3(x)))
        x= self.pool(f.relu(self.conv4(x)))
        x= self.pool(f.relu(self.conv5(x)))

        x.reshape(-1, 20*4*4)
        x= self.dropout(f.relu(self.fc1(x)))
        x= self.dropout(f.relu(self.fc2(x)))
        x= self.dropout(f.relu(self.fc3(x)))
        x= self.dropout(f.relu(self.fc4(x)))
        x= self.fc5(x)
        return x

model =  Model().to(device)
print(model)

Model(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (conv1): Conv2d(3, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(12, 14, kernel_size=(4, 4), stride=(1, 1))
  (conv4): Conv2d(14, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv5): Conv2d(16, 20, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=250, bias=True)
  (fc2): Linear(in_features=250, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
  (fc5): Linear(in_features=10, out_features=2, bias=True)
)


----------OR-----------

In [ ]:
# using pretrained model
from torchvision import models
from collections import OrderedDict

model = models.densenet121(pretrained=True)

model = model.to(device)

model = models.densenet121(pretrained=True)
    #
for param in model.parameters():
    param.requires_grad = False
    model.classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(1024, 512)),
                              ('relu', nn.ReLU()),
                              ('dropout', nn.Dropout(0.2)),
                              ('fc2', nn.Linear(512, 102)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))

model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [ ]:
#training
model.train()

for epoch in range(15):
    total_correct = 0
    running_loss = 0
    for i, (inputs,labels) in enumerate(train_set):
        inputs,labels = inputs.to(device) , labels.to(device)
        output = model(inputs)
        output_idx = torch.argmax(output, dim=1)
        total_correct += (labels == output_idx).sum().item()
        optimizer.zero_grad()
        loss = criterion(output,labels)
        running_loss += loss.item() * inputs.size(0)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch} Loss: {running_loss / train_len} Accuracy: {(total_correct / train_len) * 100}%')

print('finished training')

In [ ]:
#testing the model

with torch.no_grad():
 model.eval()
 total_loss =0
 total_correct =0

for inputs,labels in test_set:
    labels= labels.to(device)
    outputs = model(inputs.to(device))
    loss = criterion(outputs,labels)
    total_loss += loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs, dim=1)
    total_correct += sum(labels == output_idx)

print(f'Accuracy : {(total_correct/test_len)*100}% Loss:{total_loss/test_len}')

In [ ]:
#saving the model
torch.save(model.state_dict() , 'dogclassifier.pt')

In [ ]:
#loading the model
with torch.no_grad():
    model = model().to(device)
    model.load_state_dict(torch.load('dogclassifier.pt'))
    model.eval()